In [6]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os

def process_multiple_inputs(input_file_output_folder_pairs):
    """
    Processes multiple input file pairs, computes the classification report and confusion matrix,
    and saves the outputs (confusion matrix image and classification report CSV) to the respective output folders.

    Args:
        input_file_output_folder_pairs (list of tuples): A list where each tuple contains:
            - First element: List of two file paths [predictions_file, labels_file].
            - Second element: Output folder path for saving results.

    Returns:
        None
    """
    # Define the column headers and label mapping
    columns = ['Mild Pain', 'No Pain', 'Pain']
    label_mapping = {0: 'No Pain', 1: 'Mild Pain', 2: 'Obvious Pain'}

    for file_pair in input_file_output_folder_pairs:
        # Unpack each tuple
        input_files, output_folder = file_pair
        predictions_file = input_files[0]
        labels_file = input_files[1]

        # Ensure the output folder exists
        os.makedirs(output_folder, exist_ok=True)

        # Read and process labels
        with open(labels_file, 'r') as lab:
            labels = lab.readlines()
            labels = [x.strip() for x in labels]
            labels = [x.split(' ') for x in labels]
            labels = pd.DataFrame(labels, columns=columns)
            labels = labels.astype(int).idxmax(axis=1)  # Convert one-hot to class labels

        # Read and process predictions
        with open(predictions_file, 'r') as pred:
            predictions = pred.readlines()
            predictions = [x.strip() for x in predictions]
            predictions = [x.split(' ') for x in predictions]
            predictions = pd.DataFrame(predictions, columns=columns)
            predictions = predictions.astype(int).idxmax(axis=1)  # Convert one-hot to class labels

        # Generate and save classification report
        report = classification_report(labels, predictions, target_names=label_mapping.values(), output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_csv_path = os.path.join(output_folder, 'classification_report.csv')
        report_df.to_csv(report_csv_path)
        print(f"Classification report saved at: {report_csv_path}")

        # Generate and save confusion matrix
        cm = confusion_matrix(labels, predictions)
        plt.figure(figsize=(7, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='viridis',
                    xticklabels=label_mapping.values(), yticklabels=label_mapping.values(), annot_kws={"size": 14})
        # Customizing visualization
        plt.xlabel("Predicted", fontsize=16, fontweight='bold')
        plt.ylabel("Actual", fontsize=16, fontweight='bold')
        plt.xticks(fontsize=14)  # Set x-ticklabel font size
        plt.yticks(fontsize=14)  # Set y-ticklabel font size
        # No title
        confusion_matrix_path = os.path.join(output_folder, 'confusion_matrix.png')
        plt.savefig(confusion_matrix_path, bbox_inches='tight', pad_inches=0.1)
        plt.close()  # Close the plot to avoid memory issues
        print(f"Confusion matrix saved at: {confusion_matrix_path}")
# Example list of input file pairs
input_file_output_folder_pairs = [
    (['pain_predictions_only_bb.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'only backbone'),
    (['pain_predictions_no_gnn.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'no gnn'),
    (['pain_predictions.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'full'),
    (['full gr.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'full + graph representation'),
    (['full gr no sft.txt', '../data/UNBC/list/UNBC_test_pspi_fold1.txt'], 'full no sft'),
]

# Call the function
process_multiple_inputs(input_file_output_folder_pairs)

Classification report saved at: only backbone\classification_report.csv
Confusion matrix saved at: only backbone\confusion_matrix.png
Classification report saved at: no gnn\classification_report.csv
Confusion matrix saved at: no gnn\confusion_matrix.png
Classification report saved at: full\classification_report.csv
Confusion matrix saved at: full\confusion_matrix.png
Classification report saved at: full + graph representation\classification_report.csv
Confusion matrix saved at: full + graph representation\confusion_matrix.png
Classification report saved at: full no sft\classification_report.csv
Confusion matrix saved at: full no sft\confusion_matrix.png
